### Recruit Visitor Forecasting - Gradient Boosted Trees (with lgbm)

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [17]:
#Contibutions from: DSEverything - Mean Mix - Math, Geo, Harmonic
#https://www.kaggle.com/dongxu027/mean-mix-math-geo-harmonic-lb-0-493 
#JdPaletto - Surprised Yet? - Part2
#https://www.kaggle.com/jdpaletto/surprised-yet-part2-lb-0-503
#hklee - weighted mean comparisons
#https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st

#Load required libraries
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
import lightgbm as lgb

In [3]:
data = {
    'tra': pd.read_csv('air_visit_data.csv'),
    'as': pd.read_csv('air_store_info.csv'),
    'hs': pd.read_csv('hpg_store_info.csv'),
    'ar': pd.read_csv('air_reserve.csv'),
    'hr': pd.read_csv('hpg_reserve.csv'),
    'id': pd.read_csv('store_id_relation.csv'),
    'tes': pd.read_csv('sample_submission.csv'),
    'hol': pd.read_csv('date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

In [4]:
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])
data['hr'].head(5)

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926


In [5]:
for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

data['ar'].head(5)
data['hr'].head(5)

,air_store_id,visit_date,rs1,rv1,rs2,rv2
0,air_00a91d42b08b08d9,2016-10-31,0,2,0.0,2.0
1,air_00a91d42b08b08d9,2016-12-05,4,9,4.0,9.0
2,air_00a91d42b08b08d9,2016-12-14,6,18,6.0,18.0
3,air_00a91d42b08b08d9,2016-12-17,6,2,6.0,2.0
4,air_00a91d42b08b08d9,2016-12-20,2,4,2.0,4.0


,air_store_id,visit_date,rs1,rv1,rs2,rv2
0,air_00a91d42b08b08d9,2016-01-14,3,2,3.0,2.0
1,air_00a91d42b08b08d9,2016-01-15,6,4,6.0,4.0
2,air_00a91d42b08b08d9,2016-01-16,3,2,3.0,2.0
3,air_00a91d42b08b08d9,2016-01-22,3,2,3.0,2.0
4,air_00a91d42b08b08d9,2016-01-29,6,5,6.0,5.0


In [6]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

data['tra'].head(5)
data['tes'].head(5)

unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

print(unique_stores)
stores.head(5)

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1


,id,visitors,visit_date,air_store_id,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4


['air_00a91d42b08b08d9' 'air_0164b9927d20bcc3' 'air_0241aa3964b7f861'
 'air_0328696196e46f18' 'air_034a3d5b40d5b1b1' 'air_036d4f1ee7285390'
 'air_0382c794b73b51ad' 'air_03963426c9312048' 'air_04341b588bde96cd'
 'air_049f6d5b402a31b2' 'air_04cae7c1bc9b2a0b' 'air_0585011fa179bcce'
 'air_05c325d315cc17f5' 'air_0647f17b4dc041c8' 'air_064e203265ee5753'
 'air_066f0221b8a4d533' 'air_06f95ac5c33aca10' 'air_0728814bd98f7367'
 'air_0768ab3910f7967f' 'air_07b314d83059c4d2' 'air_07bb665f9cdfbdfb'
 'air_082908692355165e' 'air_083ddc520ea47e1e' 'air_0845d8395f30c6bb'
 'air_084d98859256acf0' 'air_0867f7bebad6a649' 'air_08ba8cd01b3ba010'
 'air_08cb3c4ee6cd6a22' 'air_08ef81d5b7a0d13f' 'air_08f994758a1e76d4'
 'air_09040f6df960ddb8' 'air_0919d54f0c9a24b8' 'air_09661c0f3259cc04'
 'air_09a845d5b5944b01' 'air_09fd1f5c58583141' 'air_0a74a5408a0b8642'
 'air_0b184ec04c741a6a' 'air_0b1e72d2d4422b20' 'air_0b9038300f8b2b50'
 'air_0e1eae99b8723bc1' 'air_0e7c11b9abc50163' 'air_0f0cdeee6c9bf3d7'
 'air_0f2f96335f2748

,air_store_id,dow
0,air_00a91d42b08b08d9,0
1,air_0164b9927d20bcc3,0
2,air_0241aa3964b7f861,0
3,air_0328696196e46f18,0
4,air_034a3d5b40d5b1b1,0


In [7]:
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
stores.head(5)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0,Dining bar,Ōsaka-fu Ōsaka-shi Nakanochō,34.701279,135.528090
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229


In [8]:
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

stores.head(5)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,...,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0,6,44,35.694003,...,0,0,0,0,0,0,0,0,0,0
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0,6,62,35.658068,...,0,0,0,0,0,0,0,0,0,0
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0,7,82,35.712607,...,0,0,0,0,0,0,0,0,0,0
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0,4,98,34.701279,...,0,0,0,0,0,0,0,0,0,0
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0,2,102,34.692337,...,0,0,0,0,0,0,0,0,0,0


In [9]:
list(stores)

['air_store_id',
 'dow',
 'min_visitors',
 'mean_visitors',
 'median_visitors',
 'max_visitors',
 'count_observations',
 'air_genre_name',
 'air_area_name',
 'latitude',
 'longitude',
 'air_genre_name0',
 'air_area_name0',
 'air_genre_name1',
 'air_area_name1',
 'air_genre_name2',
 'air_area_name2',
 'air_genre_name3',
 'air_area_name3',
 'air_genre_name4',
 'air_area_name4',
 'air_genre_name5',
 'air_area_name5',
 'air_genre_name6',
 'air_area_name6',
 'air_genre_name7',
 'air_area_name7',
 'air_genre_name8',
 'air_area_name8',
 'air_genre_name9',
 'air_area_name9']

In [10]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train.head(5)
test.head(5)

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,...,air_area_name9,rs1_x,rv1_x,rs2_x,rv2_x,rs1_y,rv1_y,rs2_y,rv2_y,id
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.843750,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-13
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0,2.0,20.292308,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-14
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0,4.0,34.738462,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-15
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0,6.0,27.651515,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-16
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0,2.0,13.754386,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-18


,id,visitors,visit_date,air_store_id,dow,year,month,day_of_week,holiday_flg,min_visitors,...,air_genre_name9,air_area_name9,rs1_x,rv1_x,rs2_x,rv2_x,rs1_y,rv1_y,rs2_y,rv2_y
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,3,0,2.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1,0,1.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,5,0,1.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,6,0,15.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,4,0,15.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

train.head(5)
test.head(5)

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,...,rs2_x,rv2_x,rs1_y,rv1_y,rs2_y,rv2_y,id,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.843750,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-13,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0,2.0,20.292308,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-14,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0,4.0,34.738462,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-15,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0,6.0,27.651515,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-16,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0,2.0,13.754386,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-18,NaN,NaN,NaN


,id,visitors,visit_date,air_store_id,dow,year,month,day_of_week,holiday_flg,min_visitors,...,rv1_x,rs2_x,rv2_x,rs1_y,rv1_y,rs2_y,rv2_y,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,3,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1,0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,5,0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,6,0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,4,0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

train.head(5)
test.head(5)

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,...,rv1_y,rs2_y,rv2_y,id,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.843750,...,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-13,NaN,NaN,NaN,20160113,8.362564,4.521799
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0,2.0,20.292308,...,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-14,NaN,NaN,NaN,20160114,8.362564,4.521799
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0,4.0,34.738462,...,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-15,NaN,NaN,NaN,20160115,8.362564,4.521799
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0,6.0,27.651515,...,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-16,NaN,NaN,NaN,20160116,8.362564,4.521799
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0,2.0,13.754386,...,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-18,NaN,NaN,NaN,20160118,8.362564,4.521799


,id,visitors,visit_date,air_store_id,dow,year,month,day_of_week,holiday_flg,min_visitors,...,rs1_y,rv1_y,rs2_y,rv2_y,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,3,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170423,8.326629,4.519803
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1,0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170424,8.326629,4.519803
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,5,0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170425,8.326629,4.519803
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,6,0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170426,8.326629,4.519803
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,4,0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170427,8.326629,4.519803


In [13]:
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

In [14]:
train.head(5)
test.head(5)

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,...,rv2_y,id,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,air_store_id2
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.843750,...,-1.0,air_ba937bf13d40fb24_2016-01-13,-1.0,-1.0,-1.0,20160113,8.362564,4.521799,175.409667,603
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0,2.0,20.292308,...,-1.0,air_ba937bf13d40fb24_2016-01-14,-1.0,-1.0,-1.0,20160114,8.362564,4.521799,175.409667,603
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0,4.0,34.738462,...,-1.0,air_ba937bf13d40fb24_2016-01-15,-1.0,-1.0,-1.0,20160115,8.362564,4.521799,175.409667,603
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0,6.0,27.651515,...,-1.0,air_ba937bf13d40fb24_2016-01-16,-1.0,-1.0,-1.0,20160116,8.362564,4.521799,175.409667,603
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0,2.0,13.754386,...,-1.0,air_ba937bf13d40fb24_2016-01-18,-1.0,-1.0,-1.0,20160118,8.362564,4.521799,175.409667,603


,id,visitors,visit_date,air_store_id,dow,year,month,day_of_week,holiday_flg,min_visitors,...,rs2_y,rv2_y,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,air_store_id2
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,3,0,2.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170423,8.326629,4.519803,175.447598,0
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1,0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170424,8.326629,4.519803,175.447598,0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,5,0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170425,8.326629,4.519803,175.447598,0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,6,0,15.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170426,8.326629,4.519803,175.447598,0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,4,0,15.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,20170427,8.326629,4.519803,175.447598,0


In [15]:
#RMSE function for evaluation
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [20]:
#Run models for Gradient Boosted tree, lgbm and knn
params = {}
params['application'] = 'regression'
params['boosting'] = 'gbdt'
params['learning_rate'] = 0.01
params['num_leaves'] = 32
params['min_sum_hessian_in_leaf'] = 1e-2
params['min_gain_to_split'] = 0
params['bagging_fraction'] = 0.8
params['feature_fraction'] = 0.8
params['num_threads'] = 4
params['metric'] = 'rmse'

from sklearn.model_selection import train_test_split
X_train, X_valid = train_test_split(train, test_size=0.05, random_state=42, shuffle=False)

X = X_train[col]
y = np.log1p(X_train['visitors'].values)
d_train = lgb.Dataset(X, y)

X = X_valid[col]
y = np.log1p(X_valid['visitors'].values)
d_valid = lgb.Dataset(X, y)

watchlist = [d_train, d_valid]

model1 = ensemble.GradientBoostingRegressor(learning_rate=0.1, random_state=3, n_estimators=200, subsample=0.8, 
                      max_depth =10)
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model3 = lgb.train(params, train_set=d_train, num_boost_round=50000, valid_sets=watchlist,
verbose_eval=10)

model1.fit(train[col], np.log1p(train['visitors'].values))
model2.fit(train[col], np.log1p(train['visitors'].values))

preds1 = model1.predict(train[col])
preds2 = model2.predict(train[col])
preds3 = model3.predict(train[col])

print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds1))
print('RMSE KNeighborsRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds2))
print('RMSE LGBMRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds3))

[10]	training's rmse: 0.76324	valid_1's rmse: 0.765292
[20]	training's rmse: 0.724673	valid_1's rmse: 0.730534
[30]	training's rmse: 0.691867	valid_1's rmse: 0.701307
[40]	training's rmse: 0.663143	valid_1's rmse: 0.675826
[50]	training's rmse: 0.638571	valid_1's rmse: 0.653839
[60]	training's rmse: 0.617639	valid_1's rmse: 0.635325
[70]	training's rmse: 0.599861	valid_1's rmse: 0.619599
[80]	training's rmse: 0.58482	valid_1's rmse: 0.606347
[90]	training's rmse: 0.572156	valid_1's rmse: 0.595299
[100]	training's rmse: 0.561606	valid_1's rmse: 0.586079
[110]	training's rmse: 0.552616	valid_1's rmse: 0.578308
[120]	training's rmse: 0.545062	valid_1's rmse: 0.571789
[130]	training's rmse: 0.538733	valid_1's rmse: 0.566275
[140]	training's rmse: 0.533507	valid_1's rmse: 0.561763
[150]	training's rmse: 0.529032	valid_1's rmse: 0.557898
[160]	training's rmse: 0.525255	valid_1's rmse: 0.554713
[170]	training's rmse: 0.522054	valid_1's rmse: 0.55195
[180]	training's rmse: 0.519375	valid_1's r

[1450]	training's rmse: 0.48509	valid_1's rmse: 0.53012
[1460]	training's rmse: 0.485006	valid_1's rmse: 0.530107
[1470]	training's rmse: 0.484939	valid_1's rmse: 0.53013
[1480]	training's rmse: 0.484856	valid_1's rmse: 0.530124
[1490]	training's rmse: 0.484768	valid_1's rmse: 0.530152
[1500]	training's rmse: 0.484679	valid_1's rmse: 0.530168
[1510]	training's rmse: 0.484596	valid_1's rmse: 0.530157
[1520]	training's rmse: 0.484513	valid_1's rmse: 0.530153
[1530]	training's rmse: 0.484423	valid_1's rmse: 0.530141
[1540]	training's rmse: 0.484334	valid_1's rmse: 0.530133
[1550]	training's rmse: 0.484265	valid_1's rmse: 0.530112
[1560]	training's rmse: 0.48416	valid_1's rmse: 0.530108
[1570]	training's rmse: 0.48408	valid_1's rmse: 0.530109
[1580]	training's rmse: 0.484019	valid_1's rmse: 0.530107
[1590]	training's rmse: 0.483949	valid_1's rmse: 0.530088
[1600]	training's rmse: 0.483865	valid_1's rmse: 0.530092
[1610]	training's rmse: 0.483806	valid_1's rmse: 0.530082
[1620]	training's r

[2870]	training's rmse: 0.47565	valid_1's rmse: 0.529395
[2880]	training's rmse: 0.475588	valid_1's rmse: 0.529394
[2890]	training's rmse: 0.475526	valid_1's rmse: 0.529379
[2900]	training's rmse: 0.47548	valid_1's rmse: 0.529378
[2910]	training's rmse: 0.475435	valid_1's rmse: 0.529375
[2920]	training's rmse: 0.475374	valid_1's rmse: 0.529375
[2930]	training's rmse: 0.475323	valid_1's rmse: 0.529368
[2940]	training's rmse: 0.475268	valid_1's rmse: 0.529357
[2950]	training's rmse: 0.475204	valid_1's rmse: 0.529354
[2960]	training's rmse: 0.475151	valid_1's rmse: 0.529356
[2970]	training's rmse: 0.475079	valid_1's rmse: 0.529359
[2980]	training's rmse: 0.475024	valid_1's rmse: 0.529343
[2990]	training's rmse: 0.474965	valid_1's rmse: 0.529338
[3000]	training's rmse: 0.474904	valid_1's rmse: 0.529328
[3010]	training's rmse: 0.474834	valid_1's rmse: 0.529321
[3020]	training's rmse: 0.474779	valid_1's rmse: 0.529313
[3030]	training's rmse: 0.474724	valid_1's rmse: 0.529324
[3040]	training'

[4290]	training's rmse: 0.468576	valid_1's rmse: 0.529184
[4300]	training's rmse: 0.468532	valid_1's rmse: 0.529192
[4310]	training's rmse: 0.468489	valid_1's rmse: 0.529191
[4320]	training's rmse: 0.468461	valid_1's rmse: 0.52919
[4330]	training's rmse: 0.468426	valid_1's rmse: 0.529182
[4340]	training's rmse: 0.468374	valid_1's rmse: 0.529175
[4350]	training's rmse: 0.468344	valid_1's rmse: 0.529181
[4360]	training's rmse: 0.468309	valid_1's rmse: 0.529179
[4370]	training's rmse: 0.468281	valid_1's rmse: 0.529184
[4380]	training's rmse: 0.468241	valid_1's rmse: 0.529182
[4390]	training's rmse: 0.4682	valid_1's rmse: 0.529171
[4400]	training's rmse: 0.46815	valid_1's rmse: 0.529165
[4410]	training's rmse: 0.468109	valid_1's rmse: 0.52915
[4420]	training's rmse: 0.468073	valid_1's rmse: 0.529147
[4430]	training's rmse: 0.468036	valid_1's rmse: 0.529138
[4440]	training's rmse: 0.468005	valid_1's rmse: 0.52914
[4450]	training's rmse: 0.467979	valid_1's rmse: 0.529123
[4460]	training's rm

[5710]	training's rmse: 0.462947	valid_1's rmse: 0.529219
[5720]	training's rmse: 0.462916	valid_1's rmse: 0.529219
[5730]	training's rmse: 0.462887	valid_1's rmse: 0.529227
[5740]	training's rmse: 0.462846	valid_1's rmse: 0.529216
[5750]	training's rmse: 0.462817	valid_1's rmse: 0.529212
[5760]	training's rmse: 0.462774	valid_1's rmse: 0.529196
[5770]	training's rmse: 0.462739	valid_1's rmse: 0.529192
[5780]	training's rmse: 0.462703	valid_1's rmse: 0.529175
[5790]	training's rmse: 0.462664	valid_1's rmse: 0.529171
[5800]	training's rmse: 0.462621	valid_1's rmse: 0.529177
[5810]	training's rmse: 0.462575	valid_1's rmse: 0.529175
[5820]	training's rmse: 0.462536	valid_1's rmse: 0.529177
[5830]	training's rmse: 0.462498	valid_1's rmse: 0.529184
[5840]	training's rmse: 0.462463	valid_1's rmse: 0.529174
[5850]	training's rmse: 0.462425	valid_1's rmse: 0.529171
[5860]	training's rmse: 0.462395	valid_1's rmse: 0.529179
[5870]	training's rmse: 0.462358	valid_1's rmse: 0.529169
[5880]	trainin

...

[47770]	training's rmse: 0.383478	valid_1's rmse: 0.534588
[47780]	training's rmse: 0.383465	valid_1's rmse: 0.534592
[47790]	training's rmse: 0.383449	valid_1's rmse: 0.5346
[47800]	training's rmse: 0.383431	valid_1's rmse: 0.534601
[47810]	training's rmse: 0.383414	valid_1's rmse: 0.534604
[47820]	training's rmse: 0.383401	valid_1's rmse: 0.534604
[47830]	training's rmse: 0.383386	valid_1's rmse: 0.5346
[47840]	training's rmse: 0.383371	valid_1's rmse: 0.534604
[47850]	training's rmse: 0.383355	valid_1's rmse: 0.534599
[47860]	training's rmse: 0.383339	valid_1's rmse: 0.534599
[47870]	training's rmse: 0.383322	valid_1's rmse: 0.534591
[47880]	training's rmse: 0.383301	valid_1's rmse: 0.534589
[47890]	training's rmse: 0.383288	valid_1's rmse: 0.534592
[47900]	training's rmse: 0.383269	valid_1's rmse: 0.534588
[47910]	training's rmse: 0.383254	valid_1's rmse: 0.534589
[47920]	training's rmse: 0.383243	valid_1's rmse: 0.534588
[47930]	training's rmse: 0.383233	valid_1's rmse: 0.534592
[

[49170]	training's rmse: 0.381709	valid_1's rmse: 0.534659
[49180]	training's rmse: 0.381696	valid_1's rmse: 0.534656
[49190]	training's rmse: 0.381678	valid_1's rmse: 0.534657
[49200]	training's rmse: 0.381663	valid_1's rmse: 0.534661
[49210]	training's rmse: 0.381653	valid_1's rmse: 0.534657
[49220]	training's rmse: 0.381641	valid_1's rmse: 0.534662
[49230]	training's rmse: 0.381629	valid_1's rmse: 0.534659
[49240]	training's rmse: 0.381615	valid_1's rmse: 0.534656
[49250]	training's rmse: 0.381598	valid_1's rmse: 0.53466
[49260]	training's rmse: 0.381583	valid_1's rmse: 0.534659
[49270]	training's rmse: 0.381564	valid_1's rmse: 0.534662
[49280]	training's rmse: 0.381547	valid_1's rmse: 0.534658
[49290]	training's rmse: 0.381527	valid_1's rmse: 0.534658
[49300]	training's rmse: 0.381516	valid_1's rmse: 0.53466
[49310]	training's rmse: 0.381502	valid_1's rmse: 0.534659
[49320]	training's rmse: 0.38149	valid_1's rmse: 0.534661
[49330]	training's rmse: 0.381479	valid_1's rmse: 0.53466
[

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=200, presort='auto', random_state=3,
             subsample=0.8, verbose=0, warm_start=False)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
          weights='uniform')

RMSE GradientBoostingRegressor:  0.39762162831
RMSE KNeighborsRegressor:  0.419451716114
RMSE LGBMRegressor:  0.389782127403


In [21]:
preds1t = model1.predict(test[col])
preds2t = model2.predict(test[col])
preds3t = model3.predict(test[col])

test['visitors'] = 0.5*preds1t+0.5*preds3t
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub1 = test[['id','visitors']].copy()
sub1.head(5)
test['visitors'] = 0.3*preds1t+0.3*preds2t+0.4*preds3t
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub2 = test[['id','visitors']].copy()
sub2.head(5)

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,2.103900
1,air_00a91d42b08b08d9_2017-04-24,20.934192
2,air_00a91d42b08b08d9_2017-04-25,30.482635
3,air_00a91d42b08b08d9_2017-04-26,26.992753
4,air_00a91d42b08b08d9_2017-04-27,28.172828


,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,2.449946
1,air_00a91d42b08b08d9_2017-04-24,22.545993
2,air_00a91d42b08b08d9_2017-04-25,30.828327
3,air_00a91d42b08b08d9_2017-04-26,27.408919
4,air_00a91d42b08b08d9_2017-04-27,29.669246


In [22]:
#Getting csvs for submission
sub1[['id', 'visitors']].to_csv('lgbsubmission1.csv', index=False)
sub2[['id', 'visitors']].to_csv('lgbsubmission2.csv', index=False)

In [23]:
#Adding mean weighted results to ensemble and checking (optional)

air_visit_data = pd.read_csv('air_visit_data.csv')
air_store_info = pd.read_csv('air_store_info.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
air_reserve = pd.read_csv('air_reserve.csv')
hpg_reserve = pd.read_csv('hpg_reserve.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')
sample_submission = pd.read_csv('sample_submission.csv')
date_info = pd.read_csv('date_info.csv')

In [24]:
wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True)

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), 
    how='left')['visitors_y'].values

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values

sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)

In [25]:
sub3 = sample_submission[['id', 'visitors']].copy()
sub4= pd.merge(sub1, sub3, on='id', how='inner')
sub5= pd.merge(sub2, sub3, on='id', how='inner')

In [26]:
sub4['visitors'] = 0.7*sub4['visitors_x'] + 0.3*sub4['visitors_y']* 1.1
sub5['visitors'] = 0.7*sub5['visitors_x'] + 0.3*sub5['visitors_y']* 1.1
sub4[['id', 'visitors']].to_csv('lgbsubmission3.csv', index=False)
sub5[['id', 'visitors']].to_csv('lgbsubmission4.csv', index=False)